# Fine-Tune DistilBERT for News Bias Detection

This notebook demonstrates how to fine-tune the `distilbert-base-uncased` transformer model to detect bias in news articles using the AllSides, MBFC, and BASIL datasets. The workflow is designed for Google Colab to leverage free GPU resources.

## 1. Set Up Google Colab Environment

Make sure you are running this notebook on Google Colab with GPU acceleration enabled. Go to `Runtime > Change runtime type` and select `GPU`.

## 2. Install Required Libraries

Install Hugging Face Transformers, Datasets, and other dependencies.

**Description:** This cell installs all required Python libraries for model training and data processing, including Hugging Face Transformers, Datasets, pandas, numpy, scikit-learn, and torch.

In [ ]:
# Install required libraries
!pip install transformers datasets pandas numpy scikit-learn torch nlpaug sacremoses --quiet

## 3. Import Libraries

Import all necessary Python libraries for data processing and model training.

**Description:** This cell imports all necessary Python libraries for data processing, model training, and evaluation, including torch, transformers, datasets, pandas, numpy, and scikit-learn.

In [ ]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
from nlpaug.util import Action

## 4. Download and Prepare Datasets (AllSides, MBFC, BASIL)

Download the datasets and load them into pandas DataFrames. You may need to upload files manually or use direct download links if available.

**Description:** This cell provides instructions and sample code to upload or download the AllSides, MBFC, and BASIL datasets, and loads them into pandas DataFrames for further processing.

**Available Datasets Description:**

1. **LIAR Dataset**
   - Political statements with bias indicators
   - We'll focus on statements' ideological lean (left/right) rather than party affiliation
   - Useful for political bias detection
   
2. **News Category Dataset**
   - News headlines and categories
   - We'll use categories and content to detect ideological bias
   - Focus on political and ideological content

3. **AG News Dataset**
   - Contains world and political news
   - We'll analyze content for left/right ideological markers
   - Large-scale dataset with diverse sources

**Note on Bias Labels:**
- Labels will be binary: 0 for right-leaning, 1 for left-leaning bias
- Classification based on content analysis, not party affiliation
- Uses common markers of left/right ideological positions

In [ ]:
# Create datasets directory
import os
import zipfile
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

os.makedirs('datasets', exist_ok=True)

datasets = {}

def download_file(url, filename):
    """Download file with progress bar"""
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

    with open(filename, 'wb') as f:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            f.write(data)
    progress_bar.close()

# 1. LIAR Dataset
print("\nDownloading LIAR dataset...")
try:
    liar_url = "https://www.cs.ucsb.edu/~william/data/liar_dataset.zip"
    download_file(liar_url, "datasets/liar.zip")

    with zipfile.ZipFile("datasets/liar.zip", 'r') as zip_ref:
        zip_ref.extractall("datasets/liar")

    # Load LIAR dataset
    liar_train = pd.read_csv('datasets/liar/train.tsv', sep='\t',
                            names=['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                  'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts',
                                  'pants_on_fire_counts', 'context'])
    datasets['liar'] = liar_train
    print("LIAR dataset loaded successfully")
except Exception as e:
    print(f"Error loading LIAR dataset: {str(e)}")

# 2. News Category Dataset
print("\nDownloading News Category Dataset...")
try:
    news_url = "https://raw.githubusercontent.com/rmisra/news-headlines-dataset/master/news_category_dataset_v2.json"
    response = requests.get(news_url)
    with open("datasets/news_category.json", 'w') as f:
        f.write(response.text)

    news_cat_df = pd.read_json('datasets/news_category.json', lines=True)
    datasets['news_category'] = news_cat_df
    print("News Category dataset loaded successfully")
except Exception as e:
    print(f"Error loading News Category dataset: {str(e)}")

# 3. AG News Dataset (as replacement for BABE dataset)
print("\nDownloading AG News Dataset...")
try:
    from datasets import load_dataset
    ag_news = load_dataset("ag_news")
    ag_news_df = pd.DataFrame({
        'text': ag_news['train']['text'],
        'label': ag_news['train']['label']
    })
    datasets['ag_news'] = ag_news_df
    print("AG News dataset loaded successfully")
except Exception as e:
    print(f"Error loading AG News dataset: {str(e)}")

# Clean up zip files
for file in os.listdir('datasets'):
    if file.endswith('.zip'):
        os.remove(os.path.join('datasets', file))

# Print dataset statistics
print("\nDatasets loaded successfully!")
print(f"Total number of samples in each dataset:")
for name, df in datasets.items():
    print(f"{name}: {len(df)} samples")

print("\nSample from each dataset:")
for name, df in datasets.items():
    print(f"\n{name.upper()} sample:")
    print(df.head(1))

## 5. Preprocess Data for Model Training

Clean and preprocess the datasets, encode labels, and split into train, validation, and test sets.

In [ ]:
# Data agumentation

In [ ]:
def augment_text(text, num_aug=1):
    """Augment a single text using faster techniques"""
    # Initialize synonym augmenter
    synonym_aug = naw.SynonymAug(
        aug_src='wordnet',
        aug_p=0.2  # Only replace 20% of words for speed
    )

    try:
        # Generate one augmented version
        aug_text = synonym_aug.augment(text, n=1)[0]
        return [aug_text]
    except:
        return []

def augment_dataset(df, sample_fraction=0.3):
    """Augment only a fraction of the dataset for speed"""
    augmented_data = []

    # Process each class separately
    for label in [0, 1]:
        class_df = df[df['label'] == label]
        # Sample only a fraction of the data
        sample_size = int(len(class_df) * sample_fraction)
        sampled_df = class_df.sample(n=sample_size, random_state=42)

        # Add all original samples
        augmented_data.extend([{
            'text': row['text'],
            'label': row['label']
        } for _, row in class_df.iterrows()])

        # Add augmented versions only for the sampled subset
        for _, row in sampled_df.iterrows():
            aug_texts = augment_text(row['text'])
            augmented_data.extend([{
                'text': aug_text,
                'label': row['label']
            } for aug_text in aug_texts])

    aug_df = pd.DataFrame(augmented_data)
    return aug_df.sample(frac=1, random_state=42).reset_index(drop=True)

**Description:** This cell cleans and preprocesses the datasets, encodes bias labels, and splits the combined data into training, validation, and test sets for model training.

In [ ]:
# Preprocessing function for our datasets
def preprocess_datasets(datasets):
    processed_data = []

    # Helper function to detect ideological lean
    def get_ideological_lean(text, subject=None, context=None):
        # Keywords associated with left/right ideological positions
        left_keywords = ['progressive', 'liberal', 'social justice', 'equality', 'regulation',
                        'climate change', 'gun control', 'universal healthcare']
        right_keywords = ['conservative', 'traditional', 'free market', 'deregulation',
                         'small government', 'religious freedom', 'second amendment']

        text = text.lower()
        left_count = sum(1 for word in left_keywords if word in text)
        right_count = sum(1 for word in right_keywords if word in text)

        # Consider context if available
        if context:
            context = str(context).lower()
            left_count += sum(1 for word in left_keywords if word in context)
            right_count += sum(1 for word in right_keywords if word in context)

        # If subject is available, use it for additional context
        if subject:
            subject = str(subject).lower()
            left_count += sum(1 for word in left_keywords if word in subject)
            right_count += sum(1 for word in right_keywords if word in subject)

        # Return 1 for left-leaning, 0 for right-leaning
        return 1 if left_count > right_count else 0

    # Process LIAR dataset
    if 'liar' in datasets:
        liar_df = datasets['liar']
        liar_processed = pd.DataFrame({
            'text': liar_df['statement'],
            'label': liar_df.apply(lambda row: get_ideological_lean(
                row['statement'],
                row['subject'],
                row['context']
            ), axis=1)
        }).dropna()
        processed_data.append(liar_processed)

    # Process News Category dataset
    if 'news_category' in datasets:
        news_df = datasets['news_category']
        # Focus on political/ideological content
        political_news = news_df[news_df['category'].isin(['POLITICS', 'WORLDPOST', 'WORLD NEWS'])]
        news_processed = pd.DataFrame({
            'text': political_news['headline'] + ' ' + political_news['short_description'],
            'label': political_news.apply(lambda row: get_ideological_lean(
                row['headline'] + ' ' + row['short_description']
            ), axis=1)
        }).dropna()
        processed_data.append(news_processed)

    # Process AG News dataset
    if 'ag_news' in datasets:
        ag_df = datasets['ag_news']
        # AG News labels: 1=World, 2=Sports, 3=Business, 4=Sci/Tech
        # Focus on World and Business news
        political_ag = ag_df[ag_df['label'].isin([0, 2])]  # World and Business
        ag_processed = pd.DataFrame({
            'text': political_ag['text'],
            'label': political_ag.apply(lambda row: get_ideological_lean(
                row['text']
            ), axis=1)
        }).dropna()
        processed_data.append(ag_processed)

    # Combine all processed datasets
    combined_df = pd.concat(processed_data, ignore_index=True)

    # Add augmentation
    print("Augmenting training data...")
    combined_df = augment_dataset(combined_df)
    print(f"Dataset size after augmentation: {len(combined_df)}")


    # Ensure balanced dataset
    min_samples = min(combined_df['label'].value_counts())
    left_samples = combined_df[combined_df['label'] == 1].sample(n=min_samples, random_state=42)
    right_samples = combined_df[combined_df['label'] == 0].sample(n=min_samples, random_state=42)
    balanced_df = pd.concat([left_samples, right_samples], ignore_index=True)

    # Split into train/val/test
    train_df, temp_df = train_test_split(balanced_df, test_size=0.2, random_state=42, stratify=balanced_df['label'])
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

    print(f"Training samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")
    print(f"Test samples: {len(test_df)}")

    return train_df, val_df, test_df

# Preprocess our datasets
train_df, val_df, test_df = preprocess_datasets(datasets)

# Display class distribution
print("\nClass distribution (0=right-leaning, 1=left-leaning):")
print("\nTraining set:")
print(train_df['label'].value_counts(normalize=True))
print("\nValidation set:")
print(val_df['label'].value_counts(normalize=True))
print("\nTest set:")
print(test_df['label'].value_counts(normalize=True))

# Display some examples
print("\nSample texts and their labels:")
for bias in [0, 1]:
    print(f"\n{'Right' if bias == 0 else 'Left'}-leaning examples:")
    print(train_df[train_df['label'] == bias]['text'].head(2).to_list())

## 6. Load DistilBERT Model and Tokenizer

Load the `distilbert-base-uncased` model and tokenizer from Hugging Face Transformers.

**Description:** This cell loads the DistilBERT tokenizer and sequence classification model from Hugging Face Transformers, preparing them for fine-tuning on the bias detection task.

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

## 7. Prepare Data Loaders

Tokenize the text data and create PyTorch DataLoader objects for efficient batching during training.

**Description:** This cell tokenizes the text data and converts the train, validation, and test sets into Hugging Face Dataset objects, setting them up for efficient batching and training with PyTorch.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'],
                    truncation=True,
                    padding='max_length',
                    max_length=256)

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print("Datasets prepared for training:")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


# Layer Freezing for DistilBERT Fine-tuning

Dataset Size: working with a relatively small dataset (< 100k examples), freezing layers helps prevent overfitting
Task Similarity: Bias detection is related to the original language understanding task, so the pre-trained weights are valuable
Training Efficiency: Freezing layers reduces training time and memory requirements


# Freezing some layers

Freeze the first 4 layers (more foundational language understanding)
Keep the top 2 layers trainable (task-specific adaptation)
Always keep the classifier layer trainable
Show you how many parameters are frozen vs trainable
This approach provides a good balance between:

Preserving learned language features
Adapting to the specific bias detection task
Training efficiency
Preventing overfitting
The exact number of layers to freeze (4 in this example) can be tuned based on your results.

In [ ]:
def freeze_layers(model, num_layers_to_freeze=4):
    """Freeze the first n transformer layers of DistilBERT"""
    # First freeze all parameters
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze layers from top to bottom
    # DistilBERT has 6 layers total
    for i in range(5, num_layers_to_freeze-1, -1):
        for param in model.distilbert.transformer.layer[i].parameters():
            param.requires_grad = True

    # Always unfreeze the classifier layer
    for param in model.classifier.parameters():
        param.requires_grad = True

    # Print trainable parameters info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.1%})")

    return model

# Apply freezing before creating the Trainer
model = freeze_layers(model, num_layers_to_freeze=4)  # Freeze first 4 layers

## 8. Fine-Tune DistilBERT Model

Set up the training loop, optimizer, and loss function to fine-tune DistilBERT on the bias detection task.

**Description:** This cell sets up the Hugging Face Trainer, defines training arguments, and starts the fine-tuning process for DistilBERT using the prepared datasets. It also defines metrics for model evaluation during training.

In [ ]:
from transformers import EarlyStoppingCallback, DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="steps",     # Evaluate at every eval_steps
    save_strategy="steps",          # Save at every save_steps
    eval_steps=50,                 # Evaluate every 100 steps
    save_steps=50,                 # Save every 100 steps
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    save_total_limit=2,
    push_to_hub=False,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }



# Add early stopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
      callbacks=[early_stopping],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)



In [ ]:
print("Starting training...")
trainer.train()

## 9. Evaluate Model Performance

Assess the model's accuracy, precision, recall, and F1 score on the validation/test set.

**Description:** This cell evaluates the trained DistilBERT model on the test set and prints out the accuracy, precision, recall, and F1 score to assess its performance on bias detection.

In [ ]:
# Evaluate on test set
test_results = trainer.evaluate(test_dataset)
print(f"Overall Test Results: {test_results}")

# Evaluate on different dataset types separately
def evaluate_by_source(trainer, dataset, source_name):
    results = trainer.evaluate(dataset)
    print(f"\nResults for {source_name}:")
    print(f"Accuracy: {results['eval_accuracy']:.4f}")
    print(f"Precision: {results['eval_precision']:.4f}")
    print(f"Recall: {results['eval_recall']:.4f}")
    print(f"F1 Score: {results['eval_f1']:.4f}")

# Create test datasets for each source
for source, df in datasets.items():
    # Process and tokenize source-specific test data
    source_df = preprocess_datasets({source: df})[2]  # Get test split
    source_dataset = Dataset.from_pandas(source_df)
    source_dataset = source_dataset.map(tokenize_function, batched=True)
    source_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # Evaluate on this source
    evaluate_by_source(trainer, source_dataset, source)

## 10. Save Trained Model for Deployment

Save the fine-tuned model and tokenizer to Google Drive or local storage for later use in a web application.

**Description:** This cell saves the fine-tuned DistilBERT model and tokenizer to your Google Drive, making them available for later use in a web application or for further inference.

In [ ]:
# Create a directory for the model
import os
from google.colab import files

# Setup save directory
save_dir = './bias_model'
os.makedirs(save_dir, exist_ok=True)

# Save model and tokenizer
print("Saving model and tokenizer...")
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Save training metrics
import json
metrics_file = os.path.join(save_dir, 'training_metrics.json')
with open(metrics_file, 'w') as f:
    json.dump(trainer.state.log_history, f)

# Create zip file
!zip -r bias_model.zip {save_dir}

# Trigger download
files.download('bias_model.zip')
print("Model download should start automatically")